In [40]:
import sys
sys.path.append('../../src')
from flow_reconstruction import FlowReconstruction
import pathlib
import numpy as np
import os
import glob
import pyvista as pv

def lf_variable_getter(variables_dict, variable, solution_path):
    outputs_path = pathlib.Path(solution_path) / 'outputs'
    inputs_path = pathlib.Path(solution_path) / 'inputs'

    if variables_dict[variable] in list(map(os.path.basename, glob.glob(f"{outputs_path}/*"))):
        solution_path = pathlib.Path(solution_path) / 'outputs'
    elif variables_dict[variable] in list(map(os.path.basename, glob.glob(f"{inputs_path}/*"))):
        solution_path = pathlib.Path(solution_path) / 'inputs'
    return np.loadtxt(solution_path / variables_dict[variable])


def get_block_recursive(block, name_index):
    for n in name_index:
        block = block[n]
    return block

def hf_variable_gatter(variables_dict, variable, solution_file):
    block = pv.read(solution_file)
    return get_block_recursive(block, variables_dict[variable])

fr = FlowReconstruction()
fr.converged_solutions_id = np.arange(1,44)
fr.dataset_root = pathlib.Path('/home/ppiper/ihtc_repository/data/experimental_planning_T0in_limit').resolve()
fr.lf_solution_path = './Q1D/'
fr.hf_solution_file = './SU2/outputs/cht_setupSU2.vtm'
fr.doe_file = pathlib.Path('/home/ppiper/ihtc_repository/data/experimental_planning_T0in_limit/doe_experimental_planning.txt').resolve()

fr.set_lf_variable_getter( lf_variable_getter, variables_dict={
    'Pressure' : 'p.txt',
    'Temperature' : 'T.txt',
    'Mach' : 'M.txt',
    'Area' : 'S.txt',
})

fr.set_hf_variable_getter( hf_variable_gatter, variables_dict={
    'Pressure' : 
        ['Zone 0 (Comp. Fluid)','Internal','Internal','Pressure'],
    'Temperature' :
        ['Zone 0 (Comp. Fluid)','Internal','Internal','Temperature'],
    'Mach' : 
        ['Zone 0 (Comp. Fluid)','Internal','Internal','Mach'] ,
    'Temperature_Solid' :
        ['Zone 1 (Solid Heat)', 'Internal', 'Internal', 'Temperature'] ,
    'Temperature_Solid_INNERWALL' : 
        ['Zone 1 (Solid Heat)', 'Boundary', 'INNERWALL', 'Temperature'] ,
    'Heat_Flux_UPPER_WALL' : 
        ['Zone 0 (Comp. Fluid)', 'Boundary','UPPER_WALL', 'Heat_Flux'] ,
})


In [45]:
lf_snapshots = fr.get_lf_snapshots(include_design_variables=True)
hf_snapshots = fr.get_hf_snapshots(include_design_variables=True)
fr.gen_lf_HDF5('lf.h5', include_design_variables=True)
fr.gen_hf_HDF5('hf.h5', include_design_variables=True)

In [48]:
import h5py

with h5py.File('hf.h5', 'r') as f:
    print(f['Temperature'])

<HDF5 dataset "Temperature": shape (69300, 43), type "<f8">


In [95]:
import pyvista as pv

for ii in range(1,44):
    mesh = pv.read(f'/home/ppiper/ihtc_repository/data/experimental_planning_T0in_limit_clear/{ii}/SU2/outputs/cht_setupSU2.vtm')

    for i in range(0,mesh.n_blocks):
        for j in range(0, mesh[i].n_blocks):
            for k in range(0, mesh[i][j].n_blocks):
                mesh[i][j][k].clear_data()

    mesh.save(f'/home/ppiper/ihtc_repository/data/experimental_planning_T0in_limit_clear/{ii}/SU2/outputs/cht_setupSU2.vtm')